# Demo — TurboPi capabilities

This notebook is a quick showcase:
- Movement (fast-sdk)
- Line sensors (fast-sdk)
- Eyes / LEDs (fast-sdk if available)
- Camera moves (fast-sdk if available; otherwise your `camera_lib.py`)
- TTS (your `tts_lib.py`)

If something isn't available on this robot image, the cell will warn and keep going.


In [ ]:
# --- Bootstrap (same in every notebook) ---
from pathlib import Path
import sys

def add_repo_root():
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'lessons').is_dir() and (p / 'common').is_dir():
            if str(p) not in sys.path:
                sys.path.insert(0, str(p))
            print('Repo root:', p)
            return p
    raise FileNotFoundError('Could not find repo root (needs lessons/ and common/)')

add_repo_root()

In [ ]:
# --- Imports + capability probes ---
import sys, time, importlib

print('Python:', sys.version.split()[0])

# fast-sdk core
try:
    from fast_hi_wonder import Motors
    motors = Motors()
    print('✔ fast-sdk: Motors')
except Exception as e:
    motors = None
    print('✖ fast-sdk Motors not available ->', e)

try:
    from fast_hi_wonder import InfraredSensors
    ir = InfraredSensors()
    print('✔ fast-sdk: InfraredSensors')
except Exception as e:
    ir = None
    print('✖ fast-sdk InfraredSensors not available ->', e)

try:
    from fast_hi_wonder import RGB
    rgb = RGB()
    print('✔ fast-sdk: RGB (eyes/LEDs)')
except Exception as e:
    rgb = None
    print('✖ fast-sdk RGB not available ->', e)

try:
    from fast_hi_wonder import Camera
    cam_fast = Camera()
    print('✔ fast-sdk: Camera')
except Exception as e:
    cam_fast = None
    print('✖ fast-sdk Camera not available ->', e)

# your libs (optional)
try:
    import camera_lib as camlib
    importlib.reload(camlib)
    cam_ros = getattr(camlib, 'cam', None) or camlib.get_camera()
    print('✔ camera_lib (ROS)')
except Exception as e:
    cam_ros = None
    print('✖ camera_lib not available ->', e)

try:
    import tts_lib as tts
    importlib.reload(tts)
    tts.warm_piper('ryan')
    print('✔ tts_lib')
except Exception as e:
    tts = None
    print('✖ tts_lib not available ->', e)


## 1) Movement demo

In [ ]:
if not motors:
    raise RuntimeError('Motors not available. Install/verify turbopi-fast-sdk on the robot.')

def stop():
    motors.reset_motors()

def forward(secs=0.4, speed=90):
    motors.set_velocity(speed, 0, 0)
    time.sleep(secs)
    stop()

def back(secs=0.4, speed=90):
    motors.set_velocity(speed, 180, 0)
    time.sleep(secs)
    stop()

def turn_left(secs=0.3, rate=40):
    motors.set_velocity(0, 0, +rate)
    time.sleep(secs)
    stop()

def turn_right(secs=0.3, rate=40):
    motors.set_velocity(0, 0, -rate)
    time.sleep(secs)
    stop()

def strafe_left(secs=0.3, speed=80):
    motors.translation(0, +speed)
    time.sleep(secs)
    stop()

def strafe_right(secs=0.3, speed=80):
    motors.translation(0, -speed)
    time.sleep(secs)
    stop()

print('Forward/back/turn/strafe…')
forward(); back(); turn_left(); turn_right(); strafe_left(); strafe_right();
print('Movement demo complete.')


## 2) Line sensor demo

In [ ]:
if not ir:
    print('No InfraredSensors available on this image.')
else:
    for _ in range(12):
        vals = ir.read_sensor_data()
        print('IR:', vals)
        time.sleep(0.2)


## 3) Eyes / LED demo (if available)

In [ ]:
def set_eyes(color):
    if not rgb:
        return False
    try:
        if hasattr(rgb, 'set_all'):
            rgb.set_all(color)
        elif hasattr(rgb, 'set'):
            rgb.set(0, color); rgb.set(1, color)
        return True
    except Exception:
        return False

if not set_eyes((255,0,0)):
    print('Eyes not available via fast-sdk on this robot image.')
else:
    time.sleep(0.25)
    set_eyes((0,255,0)); time.sleep(0.25)
    set_eyes((0,0,255)); time.sleep(0.25)
    set_eyes((0,0,0))
    print('Eyes demo complete.')


## 4) Camera head demo (fast-sdk if present, else camera_lib.py)

In [ ]:
if cam_fast:
    print('Using fast-sdk Camera')
    # We don't know the exact API on every image, so we probe common methods.
    for name in ('center_all','center','home'):
        if hasattr(cam_fast, name):
            getattr(cam_fast, name)(); break
    for name in ('nod','shake','wiggle','tiny_wiggle','glance_left','glance_right','look_up','look_down'):
        if hasattr(cam_fast, name):
            getattr(cam_fast, name)();
    print('Camera demo complete.')
elif cam_ros:
    print('Using camera_lib (ROS)')
    cam_ros.center_all()
    cam_ros.nod(depth=220, speed_s=0.15)
    cam_ros.shake(width=220, speed_s=0.15)
    cam_ros.wiggle(cycles=1, amplitude=160, speed_s=0.12)
    cam_ros.center_all()
    print('Camera demo complete.')
else:
    print('No camera control available (fast-sdk Camera or camera_lib).')


## 5) TTS demo (tts_lib.py)

In [ ]:
if not tts:
    print('tts_lib not available')
else:
    path = tts.pre_synth('Kia ora! This is a TurboPi demo from Jupyter.', voice='ryan', length_scale='0.98', sentence_silence='0.08')
    print('Playing:', path)
    tts.play_path_async(path)
    time.sleep(0.2)
    print('TTS queued')
